# Question 1

- Dr. Sophia Suarez presented on the topic of image classification, specifically on the the supervised approach of analyzing an image and assigning it a label/category
- Convolutional Neural Networks (CNNs) are a widely used model for this image classification problem. They combine convolutional layers and pooling layers to learn patterns, textures, shapes in images.
- Dr. Suarez has been working in collaboration with the Ministry of Natural Resources and Forestry to classify images of zooplankton, which are useful indicators of the health of fresh water ecosystems.
- She currently has a functioning model to classify images of the species samples. The next step for her work is to build a hierachical model, which classifies on different levels of the species hiearchy, i.e. colonial vs unicellular, spines vs no spines, etc. The purpose of this would be to build a model that allows flexible grouping of species at any classification level or resolution of choice.


# Question 2

https://towardsdatascience.com/pytorch-image-classification-tutorial-for-beginners-94ea13f56f2/

Note: 
Steps 1 through 5 are for training. Skip to step 6 to load model and run predictions. 

### 1. Create Conda Environment

conda create -n torch_env python=3.10 -y

conda activate torch_env

conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia -y

conda install numpy matplotlib pandas timm scikit-learn opencv -y

pip install albumentations albumentations[imgaug] tqdm

### 2. Import packages

In [ ]:
# Standard packages
import sys
import os
import numpy as np
import pandas as pd 
pd.set_option("display.max_colwidth", None)

# Visualization
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

# Pytorch
import torch
from torch.utils.data import DataLoader


# Image loader/reader
import cv2

# Pre-trained image model
import timm

# Sci-kit learn
from sklearn.model_selection import StratifiedKFold, train_test_split


# And custom modules
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)
from src.config import * # we get all of the model specifications
from src.dataset import CustomDataset
from src.plots import visualize_history
from src.modeling.train import fit
from src.modeling.predict import *
from src.utils import *

### 3. Data Exploration

In [ ]:
sub_folders = ["Cheetahs", "Lions"]
labels = [0, 1]

data = []

for s, l in zip(sub_folders, labels):
    folder_path = os.path.join(cfg.DATA_DIR, s)
    print(folder_path)
    for r, d, f in os.walk(folder_path):
        for file in f:
            if file.lower().endswith(".jpg"):
                data.append((os.path.join(r, file), l))

df = pd.DataFrame(data, columns=['file_name','label'])
df.shape

In [ ]:
# Data cleaning
def drop_file(file_name):
    file_name = os.path.join(cfg.DATA_DIR, file_name)
    return df[df.file_name != file_name].reset_index(drop=True)

def edit_label(file_name, new_value):
    df.loc[df.file_name == file_name, "label"] = new_value
    return df

# these are incorrectly labeled
df = edit_label("Cheetahs/055dbbcca8626dbb.jpg", 1)
df = edit_label("Cheetahs/01d688c043bdbfbb.jpg", 1)

# these are non cheetah/lion images
files = [
    "Cheetahs\\02b086c4e96396f6.jpg",
    "Lions\\0086c462d6a43b30.jpg",
    "Lions\\002c2d9952ea0b75.jpg",
    "Lions\\b551ce86336e1f38.jpg",
    "Cheetahs\\0c20fa69621a2e6c.jpg",
    "Cheetahs\\02dd8dd4344b04a7.jpg",
    "Cheetahs\\52b64d96fc0647e8.jpg",
    "Cheetahs\\02b086c4e96396f6.jpg",
    "Lions\\00076a1ba8912d87.jpg",
    "Lions\\003d62f84395408f.jpg",
    "Cheetahs\\341bedaae1e7ad89.jpg",
    "Lions\\0005dcb871947109.jpg",
    "Cheetahs\\00707659aba29334.jpg",
    "Cheetahs\\01750ba1a197e3ad.jpg",
    "Cheetahs\\b2dda5f3f0bdad53.jpg",
    "Lions\\05dcb92ec07ac597.jpg",
    "Lions\\e58cc34f71331452.jpg",
    "Cheetahs\\03e4a856df44695a.jpg",
    "Lions\\0099aeca2bc9c585.jpg",
    "Lions\\e830bd6011902b40.jpg",
    "Cheetahs\\00d100b0231b60e6.jpg",
    "Lions\\00ad4439a6573080.jpg",
    "Lions\\00fac52924506248.jpg",
    "Lions\\d153aaa9667d658a.jpg",
    "Cheetahs\\02a5846a35629f1d.jpg",
    "Cheetahs\\ebd1867dacf535c7.jpg",
    "Lions\\00c5ba7d4683eddd.jpg",
    "Lions\\0568d569fd07c96c.jpg"
]

for f in files:
    df = drop_file(f)
    

df = df.sample(frac=1, random_state=cfg.seed).reset_index(drop=True)
df.shape

In [ ]:
# check class distribution- classes are balanced!
sns.countplot(data = df, x = 'label')

In [ ]:
# plot some image samples
fig, ax = plt.subplots(2, 3, figsize=(10, 6))

idx = 0
for i in range(2):
    for j in range(3):

        label = df.label[idx]
        file_path = os.path.join(cfg.DATA_DIR, df.file_name[idx])
        # print(file_path)
        # print(i,j)

        # Read an image with OpenCV
        image = cv2.imread(file_path)

        # Convert the image to RGB color space.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Resize image
        image = cv2.resize(image, (256, 256))

        ax[i,j].imshow(image)
        ax[i,j].set_title(f"Label: {label} ({'Lion' if label == 1 else 'Cheetah'})")
        ax[i,j].axis('off')
        idx = idx+1

plt.tight_layout()
plt.show()

In [ ]:
# plot some augmented data
example_dataset = CustomDataset(cfg, df,image_size=256, transform=None,mode= "train")

example_dataloader = DataLoader(example_dataset, 
                              batch_size = 32, 
                              shuffle = True, 
                              num_workers=0,
                             )

image_batch, label_batch = next(iter(example_dataloader))
fig, ax = plt.subplots(2, 3, figsize=(10, 6))

idx = 0
for i in range(2):
    for j in range(3):

        image = image_batch[idx]
        label = label_batch[idx]

        # Reshape image
        image = image.permute(1, 2, 0)

        ax[i,j].imshow(image)
        ax[i,j].set_title(f"Image {idx}\nLabel: {label} ({'Lion' if label == 1 else 'Cheetah'})")
        ax[i,j].axis('off')
        idx = idx+1

plt.tight_layout()
plt.show()


### 4. K-fold cross validation with random search hyperparameter optimization)

In [ ]:
# Set for reproducibility
set_seed(cfg.seed)

# Keep track of best hyperparameter combination
best_val_acc = 0
best_params = None
n_random_trials = 5  # number of random hyperparameter combinations to sample

# Split data into train and test sets- hyperparameter optimization should only be done on train/validation sets
train_df, test_df = train_test_split(df, 
                                      test_size = 0.1, 
                                      random_state = cfg.seed)
train_df = train_df.reset_index(drop=True)

for trial in range(n_random_trials):
    print(f"\n=== Random Trial {trial+1}/{n_random_trials} ===")
    # Specify model
    model = timm.create_model(cfg.backbone, pretrained=cfg.pretrained, num_classes=cfg.n_classes)
    model = model.to(cfg.device)
        
    # Random sampling of hyperparameters
    sampled_hp = sample_hyperparameters(cfg)
    print("Sampled hyperparameters:", sampled_hp)
    
    # Apply sampled hyperparameters to cfg
    cfg = apply_hyperparameters(cfg, sampled_hp)
    

    fold_val_accs = []
    fold_train_loss = []
    fold_val_loss = []

    # Assign each data image to a kfold class
    skf = StratifiedKFold(n_splits=cfg.n_folds, shuffle=True,random_state=cfg.seed)
    train_df["kfold"] = -1 
    for fold, (train_, val_) in enumerate(skf.split(X = train_df, y = train_df.label)):
        train_df.loc[val_ , "kfold"] = fold


    for fold in range(cfg.n_folds):
        print(f"\n--- Fold {fold+1}/{cfg.n_folds} ---\n")

        # Split data
        train_df_fold = train_df[train_df.kfold != fold].reset_index(drop=True)
        valid_df_fold = train_df[train_df.kfold == fold].reset_index(drop=True)

        # Datasets
        train_dataset = CustomDataset(cfg, train_df_fold, image_size= cfg.image_size, mode="train")
        valid_dataset = CustomDataset(cfg, valid_df_fold, image_size= cfg.image_size,mode="val")

        # Dataloaders
        train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
        valid_loader = DataLoader(valid_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

        # Optimizer
        if cfg.optimizer_type == "Adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate, weight_decay=cfg.weight_decay, betas=cfg.betas)
            lr_min = 0.1*cfg.learning_rate
        if cfg.optimizer_type == "SGD":
            optimizer = torch.optim.SGD(model.parameters(), lr=cfg.learning_rate, momentum=cfg.momentum, weight_decay=cfg.weight_decay)
            lr_min = 0.01*cfg.learning_rate

        # Scheduler- cosine decay to adapt the learning rate during the training process
        total_steps = len(train_loader) * cfg.epochs

        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=total_steps,
            eta_min=lr_min
        )


        # Train
        acc_list, loss_list, val_acc_list, val_loss_list, model, lrs = fit(
            model, optimizer, scheduler, cfg, train_loader, valid_loader
        )

        fold_val_accs.append(val_acc_list[-1])
        fold_train_loss.append(loss_list)
        fold_val_loss.append(val_loss_list)

    # Visualize training
    visualize_history(cfg, fold_train_loss, fold_val_loss)

    avg_val_acc = np.mean(fold_val_accs)
    print(f"Trial {trial+1} Average Validation Accuracy: {avg_val_acc:.4f}")

    if avg_val_acc > best_val_acc:
        best_val_acc = avg_val_acc
        best_params = sampled_hp
    
    del model, optimizer, scheduler
    torch.cuda.empty_cache()        

print("\n=== Best Hyperparameters ===")
print("Best Params:", best_params)
print("Best Validation Accuracy:", best_val_acc)

### 5. Fine-tune on full training set with "best" hyperparameters

In [ ]:
cfg.epochs = 10
train_dataset = CustomDataset(cfg, train_df,image_size=best_params['image_size'], mode= "train")

train_dataloader = DataLoader(train_dataset, 
                          batch_size = best_params["batch_size"], 
                          shuffle = True, 
                          num_workers = 0,
                         )

model = timm.create_model(cfg.backbone, pretrained=cfg.pretrained, num_classes=cfg.n_classes)
model = model.to(cfg.device)

criterion = cfg.criterion


 # Optimizer
if cfg.optimizer_type == "Adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr_adam"], weight_decay=best_params['weight_decay'], betas=best_params['betas'])
    lr_min = 0.1*best_params["lr_adam"]
if cfg.optimizer_type == "SGD":
    optimizer = torch.optim.SGD(model.parameters(), lr=best_params["lr_sgd"], momentum=best_params['momentum'], weight_decay=best_params['weight_decay'])
    lr_min = 0.01*best_params["lr_sgd"]

# Scheduler- cosine decay to adapt the learning rate during the training process
total_steps = len(train_loader) * best_params['epochs']

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=total_steps,
    eta_min=lr_min
)



acc, loss, val_acc, val_loss, model, lrs = fit(model, optimizer, scheduler, cfg, train_dataloader)

# Save down trained model and hyperparameters
torch.save(model.state_dict(), os.path.join(cfg.OUTPUT_DIR,"cheetah_lion_classifier_weights.pth"))
torch.save(best_params, os.path.join(cfg.OUTPUT_DIR, 'cheetah_lion_classifier_hyperparameters.pth'))



### 6. Predict on test set

In [ ]:
# Split data into train and test sets- comment this out if notebook is run to train + test in one run
train_df, test_df = train_test_split(df, 
                                      test_size = 0.1, 
                                      random_state = cfg.seed)

# load model and hyperparameters
model = timm.create_model(cfg.backbone, pretrained=cfg.pretrained, num_classes=cfg.n_classes)
model = model.to(cfg.device)
state_dict = torch.load(os.path.join(cfg.OUTPUT_DIR,"cheetah_lion_classifier_weights.pth"), weights_only=True, map_location='cpu')
model.load_state_dict(state_dict)

best_params = torch.load(os.path.join(cfg.OUTPUT_DIR, 'cheetah_lion_classifier_hyperparameters.pth'))

In [ ]:
test_dataset = CustomDataset(cfg, test_df,image_size=best_params['image_size'], mode= "test")


test_dataloader = DataLoader(test_dataset, 
                          batch_size = best_params["batch_size"], 
                          shuffle = False, 
                          num_workers = 0,
                         )
predictions, metric = predict(test_dataloader, model, cfg)

In [ ]:
test_df['prediction'] = predictions
test_df =  test_df.sample(frac=1, random_state=cfg.seed).reset_index(drop=True)

for (idx, batch) in enumerate(test_dataloader):

    fig, ax = plt.subplots(2, 4, figsize=(10, 6))

    idx = 0
    for i in range(2):
        for j in range(4):

            label = batch[1][idx]
            image = batch[0][idx]
            pred = predictions[idx]

            # Reshape image
            image = image.permute(1, 2, 0)

            ax[i,j].imshow(image)
            ax[i,j].set_title(f"Ground Truth: {label} ({'Lion' if label == 1 else 'Cheetah'}) \n Prediction: {pred} ({'Lion' if pred == 1 else 'Cheetah'}) ")#\n{df.file_name[idx]}")#, fontsize=14)
            ax[i,j].axis('off')
            idx = idx+1
            
            color = 'green' if label == pred else 'red'
            ax[i,j].add_patch(Rectangle((0, 0), best_params['image_size'],best_params['image_size'],
                      alpha=1, edgecolor=color, linewidth=5, fill=None))
    plt.tight_layout()
    plt.show()

# Question 3

We will use images of Calanoid and Daphnia to train model to classify zooplankton images.

In [ ]:
sub_folders = ["Calanoid", "Daphnia"]
labels = [0, 1]

data = []

for s, l in zip(sub_folders, labels):
    folder_path = os.path.join(cfg.DATA_DIR, s)
    print(folder_path)
    for r, d, f in os.walk(folder_path):
        for file in f:
            if file.lower().endswith((".tif", ".tiff")):
                data.append((os.path.join(r, file), l))

df = pd.DataFrame(data, columns=['file_name','label'])
df.head(10)

In [ ]:
# check class distribution- classes are balanced!
sns.countplot(data = df, x = 'label')

In [ ]:
# plot some image samples
fig, ax = plt.subplots(2, 3, figsize=(10, 6))

# shuffle df
df_sample = df.sample(frac=1, random_state=123).reset_index(drop=True)

idx = 0
for i in range(2):
    for j in range(3):

        label = df_sample.label[idx]
        file_path = os.path.join(cfg.DATA_DIR, df_sample.file_name[idx])

        # Read an image with OpenCV
        image = cv2.imread(file_path)

        # Convert the image to RGB color space.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Resize image
        image = cv2.resize(image, (256, 256))

        ax[i,j].imshow(image)
        ax[i,j].set_title(f"Label: {label} ({'Daphnia' if label == 1 else 'Calanoid'})")
        ax[i,j].axis('off')
        idx = idx+1

plt.tight_layout()
plt.show()

In [ ]:
# plot some augmented data
example_dataset = CustomDataset(cfg, df,image_size=128, transform=None,mode= "train")

example_dataloader = DataLoader(example_dataset, 
                              batch_size = 1, 
                              shuffle = True, 
                              num_workers=0,
                             )

fig, ax = plt.subplots(2, 3, figsize=(10, 6))

for i in range(2):
    for j in range(3):
        image_batch, label_batch = next(iter(example_dataloader))


        image = image_batch[0]
        label = label_batch[0]

        # Reshape image
        image = image.permute(1, 2, 0)

        ax[i,j].imshow(image)
        ax[i,j].set_title(f"Label: {label} ({'Daphnia' if label == 1 else 'Calanoid'})")
        ax[i,j].axis('off')
        idx = idx+1

plt.tight_layout()
plt.show()


### 4. K-fold cross validation with random search hyperparameter optimization)

In [ ]:
# Set reproducibility
set_seed(cfg.seed)

# Keep track of best hyperparameter combination
best_val_acc = 0
best_params = None
n_random_trials = 3  # number of random hyperparameter combinations to sample

# Split data into train and test sets- hyperparameter optimization should only be done on train/validation sets
train_df, test_df = train_test_split(df, 
                                      test_size = 0.2, 
                                      random_state = 42)
train_df = train_df.reset_index(drop=True)

for trial in range(n_random_trials):
    print(f"\n=== Random Trial {trial+1}/{n_random_trials} ===")
    # Specify model
    model = timm.create_model(cfg.backbone, pretrained=cfg.pretrained, num_classes=cfg.n_classes)
    model = model.to(cfg.device)
        
    # Random sampling of hyperparameters
    sampled_hp = sample_hyperparameters(cfg)
    
    # Apply sampled hyperparameters to cfg
    cfg = apply_hyperparameters(cfg, sampled_hp)

    # Force set batch size to 1 (we have few total samples)
    cfg.batch_size = 1

    # Force set epochs to 3 (don't wan't to overtrain on small set)
    cfg.epochs = 3

    
    print("Sampled hyperparameters:", sampled_hp)

    
    fold_train_loss = []
    fold_val_loss = []
    fold_val_accs = []

    # Assign each data image to a kfold class
    skf = StratifiedKFold(n_splits=cfg.n_folds, shuffle=True,random_state=cfg.seed)
    train_df["kfold"] = -1 
    for fold, (train_, val_) in enumerate(skf.split(X = train_df, y = train_df.label)):
        train_df.loc[val_ , "kfold"] = fold


    for fold in range(cfg.n_folds):
        print(f"\n--- Fold {fold+1}/{cfg.n_folds} ---\n")

        # Split data
        train_df_fold = train_df[train_df.kfold != fold].reset_index(drop=True)
        valid_df_fold = train_df[train_df.kfold == fold].reset_index(drop=True)

        # Datasets
        train_dataset = CustomDataset(cfg, train_df_fold, image_size= cfg.image_size, mode="train")
        valid_dataset = CustomDataset(cfg, valid_df_fold, image_size= cfg.image_size,mode="val")

        # Dataloaders
        train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
        valid_loader = DataLoader(valid_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

        # Optimizer
        if cfg.optimizer_type == "Adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate, weight_decay=cfg.weight_decay, betas=cfg.betas)
            lr_min = 0.1*cfg.learning_rate
        if cfg.optimizer_type == "SGD":
            optimizer = torch.optim.SGD(model.parameters(), lr=cfg.learning_rate, momentum=cfg.momentum, weight_decay=cfg.weight_decay)
            lr_min = 0.01*cfg.learning_rate

        # Scheduler- cosine decay to adapt the learning rate during the training process
        total_steps = len(train_loader) * cfg.epochs

        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=total_steps,
            eta_min=lr_min
        )


        # Train
        acc_list, loss_list, val_acc_list, val_loss_list, model, lrs = fit(
            model, optimizer, scheduler, cfg, train_loader, valid_loader
        )

        fold_val_accs.append(val_acc_list[-1])
        fold_train_loss.append(loss_list)
        fold_val_loss.append(val_loss_list)

    # Visualize training
    visualize_history(cfg, fold_train_loss, fold_val_loss)

    avg_val_acc = np.mean(fold_val_accs)
    print(f"Trial {trial+1} Average Validation Accuracy: {avg_val_acc:.4f}")

    if avg_val_acc > best_val_acc:
        best_val_acc = avg_val_acc
        best_params = sampled_hp
    
    del model, optimizer, scheduler
    torch.cuda.empty_cache()        

print("\n=== Best Hyperparameters ===")
print("Best Params:", best_params)
print("Best Validation Accuracy:", best_val_acc)

### 5. Fine-tune on full training set with "best" hyperparameters

In [ ]:
train_dataset = CustomDataset(cfg, train_df,image_size=best_params['image_size'], mode= "train")

train_dataloader = DataLoader(train_dataset, 
                          batch_size = best_params["batch_size"], 
                          shuffle = True, 
                          num_workers = 0,
                         )

model = timm.create_model(cfg.backbone, pretrained=cfg.pretrained, num_classes=cfg.n_classes)
model = model.to(cfg.device)

criterion = cfg.criterion

 # Optimizer
if cfg.optimizer_type == "Adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr_adam"], weight_decay=best_params['weight_decay'], betas=best_params['betas'])
    lr_min = 0.1*best_params["lr_adam"]
if cfg.optimizer_type == "SGD":
    optimizer = torch.optim.SGD(model.parameters(), lr=best_params["lr_sgd"], momentum=best_params['momentum'], weight_decay=best_params['weight_decay'])
    lr_min = 0.01*best_params["lr_sgd"]

# Scheduler- cosine decay to adapt the learning rate during the training process
total_steps = len(train_loader) * best_params['epochs']

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=total_steps,
    eta_min=lr_min
)


acc, loss, val_acc, val_loss, model, lrs = fit(model, optimizer, scheduler, cfg, train_dataloader)

# Save down trained model
torch.save(model.state_dict(), os.path.join(cfg.OUTPUT_DIR,"daphnia_calanoid_classifier_weights.pth"))
torch.save(best_params, os.path.join(cfg.OUTPUT_DIR, 'daphnia_calanoid_hyperparameters.pth'))


### 6. Predict on test set

In [ ]:
# Split data into train and test sets- comment this out if notebook is run to train + test in one run
train_df, test_df = train_test_split(df, 
                                      test_size = 0.1, 
                                      random_state = cfg.seed)

# load model and hyperparameters
model = timm.create_model(cfg.backbone, pretrained=cfg.pretrained, num_classes=cfg.n_classes)
model = model.to(cfg.device)
state_dict = torch.load(os.path.join(cfg.OUTPUT_DIR,"daphnia_calanoid_classifier_weights.pth"), weights_only=True, map_location='cpu')
model.load_state_dict(state_dict)

best_params = torch.load(os.path.join(cfg.OUTPUT_DIR, 'daphnia_calanoid_classifier_hyperparameters.pth'))

In [ ]:
test_dataset = CustomDataset(cfg, test_df,image_size=best_params['image_size'], mode= "test")


test_dataloader = DataLoader(test_dataset, 
                          batch_size = best_params["batch_size"], 
                          shuffle = False, 
                          num_workers = 0,
                         )
predictions, metric = predict(test_dataloader, model, cfg)

In [ ]:
test_df['prediction'] = predictions
test_df =  test_df.sample(frac=1, random_state=123).reset_index(drop=True)

for (idx, batch) in enumerate(test_dataloader):

    fig, ax = plt.subplots(2, 3, figsize=(10, 6))

    idx = 0
    for i in range(2):
        for j in range(3):

            label = batch[1][idx]
            image = batch[0][idx]
            pred = predictions[idx]

            # Reshape image
            image = image.permute(1, 2, 0)

            ax[i,j].imshow(image)

            ax[i,j].set_title(f"Ground Truth: {label} ({'Daphnia' if label == 1 else 'Calanoid'}) \n Prediction: {pred} ({'Lion' if pred == 1 else 'Cheetah'}) ")#\n{df.file_name[idx]}")#, fontsize=14)
            ax[i,j].axis('off')
            idx = idx+1
            
            color = 'green' if label == pred else 'red'
            ax[i,j].add_patch(Rectangle((0, 0), best_params['image_size'],best_params['image_size'],
                      alpha=1, edgecolor=color, linewidth=5, fill=None))
    plt.tight_layout()
    plt.show()